In [1]:
from bs4 import BeautifulSoup
import jsonpickle
import requests

In [2]:
dataset = "food_barcodes/db.json"

In [3]:
try:
    data = jsonpickle.decode(open(dataset,'r').read())
    print("Loaded Dataset")
except FileNotFoundError:
    print("No Dataset present, defaulting to empty array")
    data = []

Loaded Dataset


In [130]:
class Food():
    def __init__(self,title, barcode, product_json):
        self.title = title
        self.barcode = barcode
        self.product_json = product_json
        self.info = {}
        for item in self.product_json['attribute_groups']:
            self.info[item['id']] = item
        self.alerts = [f"{item['title']}" for item in test.info['nutritional_quality']['attributes']]
        
    def __repr__(self):
        return f"{self.barcode}, {self.title}\nAlerts:{','.join(self.alerts)}"
    
    @staticmethod
    def FromDict(info):
        return Food(info['title'],info['barcode'],info['product_json'])

In [131]:
def lookup(code):
    res = requests.get(f"https://world.openfoodfacts.org/cgi/product.pl?code={code}",headers={"User-Agent":"Chrome v100"})
    info = {}
    soup = BeautifulSoup(res.text)
    info['barcode']=code
    info['title'] = soup.find('h1',{'itemprop':'name'}).text.strip()
    
    #get product info
    for item in soup.find_all('script'):
        if 'var product =' in str(item):
            item = str(item)
            item = item[item.find("var product = ")+len("var product = "):]
            info['product_json'] = json.loads(item[:item.find(";\n</script>")].strip())
            
    info['nova_grade'] = soup.find('div',{'class':'attribute_card'})
    
    return Food.FromDict(info)

In [134]:
scanned = []
for item in data:
    item = lookup(item['Barcode'])
    scanned.append(item)
    print(item)
    print()

038000246609, Pop tarts! Lemon creme pie
Alerts:Nutri-Score unknown,Salt in unknown quantity,Fat in unknown quantity,Sugars in unknown quantity,Saturated fat in unknown quantity

043000204726, Vanilla instant pudding pie filling mix boxes - Jell-O
Alerts:Nutri-Score unknown,Salt in unknown quantity,Fat in unknown quantity,Sugars in unknown quantity,Saturated fat in unknown quantity

072392016143, All pink strawberry gelatin
Alerts:Nutri-Score unknown,Salt in unknown quantity,Fat in unknown quantity,Sugars in unknown quantity,Saturated fat in unknown quantity

014100046691, Goldfish - Pepperidge Farm
Alerts:Nutri-Score unknown,Salt in unknown quantity,Fat in unknown quantity,Sugars in unknown quantity,Saturated fat in unknown quantity

708820075788, Dry Roasted Peanuts - Meijer
Alerts:Nutri-Score unknown,Salt in unknown quantity,Fat in unknown quantity,Sugars in unknown quantity,Saturated fat in unknown quantity

029000076501, Lightly salted dry roasted peanuts, lightly salted - Heinz
A